In [27]:
import tweepy,pymongo
import json
import pandas as pd

conskeys = 'bjFdraCy5qgh2ZN4lwZly0Ekb'
consecret = 'ORmDIHSv0676tqhzjMGmXPLlyC4DPwmp6k6wDNZPXFraj59p3e'
acctok = '593357593-P1Zxz6ZVSnuDBHcHOFy6CM8TnlOCLFLR6Jjskw25'
acctoksecret = 'tNgsfAK2QjIsXjzloq83LyMqzISHCG6puuVFvXIdtaapU'

# Authentication
auth = tweepy.OAuthHandler(conskeys, consecret)
auth.set_access_token(acctok, acctoksecret)
api = tweepy.API(auth)
query = "U.S. economy"

client = pymongo.MongoClient()
db = client["tweet_database"]
my_tweets_table = db.tweets_collection  # Setting collection to enter documents.

_max_queries = 165  # arbitrarily chosen value
tweet = tweet_batch = api.search(q=query, count=100,tweet_mode='extended')

for tweet in tweet_batch:
        tweets = []
        tweets.append((tweet.created_at,tweet.id,tweet.full_text,tweet.source, tweet.truncated, tweet.metadata, tweet.in_reply_to_status_id, tweet.in_reply_to_user_id, tweet.in_reply_to_screen_name, tweet.user._json, tweet.place,tweet.is_quote_status, tweet.retweet_count,tweet.favorite_count,tweet.entities,tweet.favorited,tweet.retweeted, tweet.lang ))
        tweetsdf = pd.DataFrame(tweets,columns=['created_at', 'id', 'full_text', 'source', 'truncated','metadata','in_reply_to_status_id','in_reply_to_user_id','in_reply_to_screen_name','user','place','is_quote_status','retweet_count','favorite_count','entities','favorited','retweeted','lang'])
        tweetsdf.to_csv('Economy_USA.csv')
        id_str = tweet.id_str
        user_id = tweet.user.id_str
        user_name = tweet.user.name
        user = tweet.user._json
        in_reply_to_user_id = tweet.in_reply_to_user_id
        lang = tweet.lang
        source = tweet.source
        retweet_count = tweet.retweet_count
        in_reply_to_status_id = tweet.in_reply_to_status_id
        try:
            x = tweet.retweeted_status  # Can't be accessed if not retweet.
            retweetedFrom = x.user.id_str
            RT = True
        except AttributeError:  # If original tweet
            retweetedFrom = -1
            RT = False
        if RT:
            content = tweet.retweeted_status.full_text
            fav = -1  # Fav count only in case of original tweets.
        else:
            content = tweet.full_text
            fav = tweet.favorite_count
        created_at = str(tweet.created_at)
        location = tweet.user.location
        ht = tweet.entities["hashtags"]
        hashtags = []
        for i in ht:
            hashtags.append(i["text"]) # List of hashtags.
        mntn = tweet.entities["user_mentions"]
        mentions = []
        for i in mntn:
            mentions.append(i['id_str']) # List of mentions
        try:
            md = tweet.entities["media"]
            media = []
            for i in md:
                media.append(i["url"]) # And list of media (photos only)
        except KeyError:  # If no media found
            media = []
        try:
            jsonx = {"id_str": id_str, "user_name": user_name, "user_id": user_id, "content": content, "created_at": created_at, "location": location,
            "hashtags": hashtags, "mentions": mentions, "in_reply_to_user_id": in_reply_to_user_id,"in_reply_to_status_id":in_reply_to_status_id, "retweetedFrom": retweetedFrom, "media": media, "retweet": RT, "FavCount": fav, "source":source, "retweet_count":retweet_count} # Making a JSON to be sent to collection.
            my_tweets_table.insert(jsonx)  # And insert
        except pymongo.errors.DuplicateKeyError as e:  # Except if it's already there
            print("Duplicate Key")
ct = 1
while ct < _max_queries:
    tweet_batch = api.search(q=query, 
                             count=100,
                             max_id=tweet_batch.max_id-1,tweet_mode='extended')
    
    for tweet in tweet_batch:
        tweets = []
        tweets.append((tweet.created_at,tweet.id,tweet.full_text,tweet.source, tweet.truncated, tweet.metadata, tweet.in_reply_to_status_id, tweet.in_reply_to_user_id, tweet.in_reply_to_screen_name, tweet.user._json, tweet.place,tweet.is_quote_status, tweet.retweet_count,tweet.favorite_count,tweet.entities,tweet.favorited,tweet.retweeted, tweet.lang ))
        tweetsdf = pd.DataFrame(tweets,columns=['created_at', 'id', 'full_text', 'source', 'truncated','metadata','in_reply_to_status_id','in_reply_to_user_id','in_reply_to_screen_name','user','place','is_quote_status','retweet_count','favorite_count','entities','favorited','retweeted','lang'])
        tweetsdf.to_csv('Economy_USA.csv', mode='a', header=False)
        id_str = tweet.id_str
        user_id = tweet.user.id_str
        user_name = tweet.user.name
        user = tweet.user._json
        in_reply_to_user_id = tweet.in_reply_to_user_id
        lang = tweet.lang
        source = tweet.source
        retweet_count = tweet.retweet_count
        in_reply_to_status_id = tweet.in_reply_to_status_id
        try:
            x = tweet.retweeted_status  # Can't be accessed if not retweet.
            retweetedFrom = x.user.id_str
            RT = True
        except AttributeError:  # If original tweet
            retweetedFrom = -1
            RT = False
        if RT:
            content = tweet.retweeted_status.full_text
            fav = -1  # Fav count only in case of original tweets.
        else:
            content = tweet.full_text
            fav = tweet.favorite_count
        created_at = str(tweet.created_at)
        location = tweet.user.location
        ht = tweet.entities["hashtags"]
        hashtags = []
        for i in ht:
            hashtags.append(i["text"]) # List of hashtags.
        mntn = tweet.entities["user_mentions"]
        mentions = []
        for i in mntn:
            mentions.append(i['id_str']) # List of mentions
        try:
            md = tweet.entities["media"]
            media = []
            for i in md:
                media.append(i["url"]) # And list of media (photos only)
        except KeyError:  # If no media found
            media = []
        try:
            jsonx = {"id_str": id_str, "user_name": user_name, "user_id": user_id, "content": content, "created_at": created_at, "location": location,
            "hashtags": hashtags, "mentions": mentions, "in_reply_to_user_id": in_reply_to_user_id,"in_reply_to_status_id":in_reply_to_status_id, "retweetedFrom": retweetedFrom, "media": media, "retweet": RT, "FavCount": fav, "source":source, "retweet_count":retweet_count} # Making a JSON to be sent to collection.
            my_tweets_table.insert(jsonx)  # And insert
        except pymongo.errors.DuplicateKeyError as e:  # Except if it's already there
            print("Duplicate Key")
    ct += 1

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:70: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:126: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [31]:
data = pd.read_csv("Economy_USA.csv")
data.head()

,Unnamed: 0,created_at,id,full_text,source,truncated,metadata,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user,place,is_quote_status,retweet_count,favorite_count,entities,favorited,retweeted,lang
0,0,2020-04-27 04:38:47,1254631052213202944,"""Investors bet on testing, treatments for rest...",Twitter for Android,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 1182422166488399872, 'id_str': '1182422...",NaN,False,0,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
1,0,2020-04-27 04:38:19,1254630936311992320,"""U.S. economy faces historic shock, with 16% j...",Twitter for Android,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 1182422166488399872, 'id_str': '1182422...",NaN,False,0,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
2,0,2020-04-27 04:38:11,1254630902895792129,"RT @tedcruz: 7/x “But over the past decade, Ch...",Twitter for Android,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 207134017, 'id_str': '207134017', 'name...",NaN,False,123,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
3,0,2020-04-27 04:38:10,1254630897556439041,RT @keyenpepper: no but do u understand how ki...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 775941300122030080, 'id_str': '77594130...",NaN,False,44,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
4,0,2020-04-27 04:37:41,1254630776022450177,RT @billstill: https://t.co/VM9MxwFr5H Did Chi...,Twitter Web App,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 86796377, 'id_str': '86796377', 'name':...",NaN,False,6,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en


In [32]:
len(data)

15332

In [22]:
data[data['id']=='1254603112066822144']

,Unnamed: 0,created_at,id,full_text,source,truncated,metadata,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user,place,is_quote_status,retweet_count,favorite_count,entities,favorited,retweeted,lang
10524,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10545,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10567,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10590,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10614,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10639,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10665,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10692,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10720,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
10749,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en


In [20]:
data[14500:14510]

,Unnamed: 0,created_at,id,full_text,source,truncated,metadata,in_reply_to_status_id,in_reply_to_user_id,in_reply_to_screen_name,user,place,is_quote_status,retweet_count,favorite_count,entities,favorited,retweeted,lang
14500,20,2020-04-27 02:47:45,1254603112066822144,RT @BuckSexton: New data suggests the lockdown...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 315295984, 'id_str': '315295984', 'name...",NaN,False,5371,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14501,21,2020-04-27 02:47:25,1254603025462595584,RT @keyenpepper: no but do u understand how ki...,Twitter Web App,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 1077008753583484928, 'id_str': '1077008...",NaN,False,43,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14502,22,2020-04-27 02:47:19,1254603002712735744,RT @politico: Dr. Anthony Fauci says the U.S. ...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 1422016674, 'id_str': '1422016674', 'na...",NaN,False,1727,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14503,23,2020-04-27 02:47:07,1254602948904173568,RT @nytimes: The coronavirus has devastated th...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 418247917, 'id_str': '418247917', 'name...",NaN,False,431,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14504,24,2020-04-27 02:45:42,1254602592526626817,"RT @tedcruz: 7/x “But over the past decade, Ch...",Twitter for iPad,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 393844737, 'id_str': '393844737', 'name...",NaN,False,119,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14505,25,2020-04-27 02:45:36,1254602568413536256,"RT @tedcruz: 7/x “But over the past decade, Ch...",Twitter Web App,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 1171619300840312832, 'id_str': '1171619...",NaN,False,119,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14506,26,2020-04-27 02:45:02,1254602428013350914,@PoliticalOpin17 @casperthemedic @realDonaldTr...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",1.2545804977720893e+18,1.254580e+18,PoliticalOpin17,"{'id': 1116182990965444608, 'id_str': '1116182...",NaN,False,0,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14507,27,2020-04-27 02:44:05,1254602188455886852,RT @NPR: Europe's economy was hit just as hard...,Twitter for iPhone,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 2842624519, 'id_str': '2842624519', 'na...",NaN,False,688,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14508,28,2020-04-27 02:44:04,1254602182403522560,RT @FAIRImmigration: The Trump Administration ...,Twitter for Android,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 1032004831379116033, 'id_str': '1032004...",NaN,False,80,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en
14509,29,2020-04-27 02:42:57,1254601902488121344,RT @NPR: Europe's economy was hit just as hard...,Twitter Web App,False,"{'iso_language_code': 'en', 'result_type': 're...",NaN,NaN,NaN,"{'id': 364563540, 'id_str': '364563540', 'name...",NaN,False,688,0,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,False,en


In [16]:
myquery = {'id_str': '1253483492320481282'}

mydoc = my_tweets_table.find(myquery)

for x in mydoc:
    print(x)

{'_id': ObjectId('5ea66527fb8f5ff7fd470ec3'), 'id_str': '1253483492320481282', 'user_name': 'Alicia Starr ⭐️🌟⭐️', 'user_id': '4870230854', 'content': 'The major American news media unbelievably wants the economy to collapse. Two reasons for that include the Trump Derangement Syndrome and the media needs eyes, regardless the cost\n\nhttps://t.co/t59gAF66xX', 'created_at': '2020-04-24 00:38:47', 'location': 'California (Bay Area)', 'hashtags': [], 'mentions': ['1174349501835948032'], 'in_reply_to_user_id': None, 'in_reply_to_status_id': None, 'retweetedFrom': '1174349501835948032', 'media': [], 'retweet': True, 'FavCount': -1, 'source': 'Twitter for iPhone', 'retweet_count': 130}


In [30]:
myquery = {}

mydoc = my_tweets_table.find(myquery)
count = 0
for x in mydoc:
    count = count + 1
print(count)

15429
